In [20]:
import pandas as pd
import pyecharts
import random
pyecharts.globals._WarningControl.ShowWarning = False
import warnings
warnings.filterwarnings('ignore')
import os
from pyecharts.components import Image
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line, Liquid, Page, Pie,Timeline
from pyecharts.commons.utils import JsCode
from pyecharts.components import Table
from pyecharts.faker import Faker


#播放数据
def sort_watch(path):   
    text = pd.read_csv(path,encoding='utf-8')
    df = text.sort_values(by=["播放量(万)"], ascending=False)
    data1 = df['播放量(万)'].tolist()
    data = df['番名'].tolist()
    ABS=[]
    for i in range(0,len(data1)):
        abs=[]
        abs.append(data[i])
        abs.append(data1[i])
        ABS.append(abs)
    return ABS 


#追番数据
def sort_follow(path):
    text = pd.read_csv(path,encoding='utf-8')
    df = text.sort_values(by=["追番人数(万)"], ascending=False)
    data1 = df['追番人数(万)'].tolist()
    data = df['番名'].tolist()
    ABS=[]
    for i in range(0,len(data1)):
        abs=[]
        abs.append(data[i])
        abs.append(data1[i])
        ABS.append(abs)
    return ABS 


def split(str):   #分离函数解决名字过长问题
    max=6
    name = '\n'.join([str[i:i+max] for i in range(0,len(str),max)])
    return name


def data_resource(data_dir):  #整合数据
    file_list = os.listdir(data_dir)   # 获取文件列表
    name_set = set()   # 存储所有人名的集合（集合中元素无重复）
    # 循环读取每个文件的数据，并将人名添加到 name_set 中
    for file_name in file_list:
        if file_name.endswith('.csv'):   # 只读取 csv 文件
            file_path = os.path.join(data_dir, file_name)
            df = pd.read_csv(file_path, header=0)
            name_set.update(set(df.iloc[:, 0]))   # 将人名添加到集合中
    # 根据 name_set 中的人名，将每个文件的数据整理成新的 DataFrame
    result_df = pd.DataFrame(columns=['Name', 'Date', 'PV', 'Follower'])   # 定义存储结果的 DataFrame
    for name in name_set:
        for file_name in file_list:
            if file_name.endswith('.csv'):   # 只读取 csv 文件
                file_path = os.path.join(data_dir, file_name)
                df = pd.read_csv(file_path, header=None)
                df_selected = df[df.iloc[:, 0] == name]   # 筛选出当前人名的行
                if len(df_selected) == 0:
                    continue   # 如果没有找到该人名，则跳过
                date = file_name[:-4]   # 取 csv 文件名中的日期
                pv = float(df_selected.iloc[0, 1])   # 取播放量
                follower = float(df_selected.iloc[0, 2])   # 取追番数
                result_df = result_df.append({'Name': name, 'Date': date, 'PV': pv, 'Follower': follower}, ignore_index=True)
    # 输出整理后的数据
    return result_df
def Image_atd(LIST) -> Image:
    #传入列表，列表内容举例['名侦探柯南（中配）', 8, 125.0, 0.0625]
    img = Image()
    img_src = (
        "./picture/"
        "{}.jpg".format(LIST[0]))
    img.add(
        src=img_src,
        style_opts={"width": "300px", "height": "400px"},
    )
    img.set_global_opts(title_opts=ComponentTitleOpts(title='{}天平均每日增长:\n播放(万):{} 追番(百):{}\n{}'
                                                       .format(LIST[1],round(LIST[2],4),round(LIST[3],4)*100,LIST[0])))
    return img
#折线图组件
def Line_atd(datas):
    #传入列表格式：datas=data.loc[data['Name'] == QB1[0][0]].values.tolist()传入的多天的数据
    l = (
        Line()
        .add_xaxis([data[1][-5:] for data in datas])
        .add_yaxis("播放量（万）", [round(data[2],1) for data in datas],color='#FFC0CB')
        .add_yaxis("追番数（百）", [round(data[3]*100,1) for data in datas],color='#39c5bb')
        .set_global_opts(yaxis_opts=opts.AxisOpts(min_='dataMin',axislabel_opts=opts.LabelOpts(font_size = 16)),
                         xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 16)),
                         title_opts=opts.TitleOpts(title="{}:({})".format(datas[0][0],len(datas)),
                                                   title_textstyle_opts=opts.LabelOpts(color='red')),
                         )
        .set_series_opts(label_opts=opts.LabelOpts(font_size=18))
            )
    return l
#QB列表里的内容，每列第一个是名字，第二个是天数，第三个是总播放量增长率，第四个是追番增长率，考虑到取消追番的因素，为负值是正常的
#计算比率（QB1综合前五，QB2取关率前五）
#冒泡排序 
def fun1(QB):
    count = len(QB)
    for i in range(count):
        for j in range(i + 1, count):
            if (QB[i][2]+QB[i][3]*50) < QB[j][2]+QB[j][3]*50:
                QB[i], QB[j] = QB[j], QB[i]
    return QB
def fun2(QB):
    count=len(QB)
    for i in range(count):
        for j in range(i + 1, count):
            if QB[i][3] > QB[j][3]:
                QB[i], QB[j] = QB[j], QB[i]
    return QB


def timeline_watch(path) -> Timeline:
    data_watch =sort_watch(path)
    name=[]
    data=[]
    for i in data_watch:
        name.append(i[0])
        data.append(i[1])
    t = Timeline()
    for i in range(0, 10):
        c = (
            Bar(init_opts=opts.InitOpts(width='2000px',
                                       height='1000px',
                                       page_title='4月29日观看总数排行'))
            .add_xaxis([split(name) for name in name[5*i:5*i+5][::-1]])
            .add_yaxis("播放人数（万）",data[5*i:5*i+5][::-1])
            .set_colors(random.choice(["red","#66ccff","#39c5bb","#FFC0CB","#fd96fd"]))
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(title_opts=opts.TitleOpts(title="播放量TOP1-50",title_link='https://www.bilibili.com/v/popular/rank/bangumi',
                                                       subtitle='数据截止到{}'.format(path[-9:-4])
                                                       ,subtitle_link='https://www.bilibili.com/v/popular/rank/bangumi/')
                            )
            )
        t.add(c,'{}-{}名'.format(5*i+1,5*(i+1)))
    return t
def timeline_follow(path) -> Timeline:
    data_follow =sort_follow(path)
    name=[]
    data=[]
    for i in data_follow:
        name.append(i[0])
        data.append(i[1])
    t = Timeline()
    for i in range(0, 10):
        c = (
            Bar()
            .add_xaxis([split(name) for name in name[5*i:5*i+5][::-1]])
            .add_yaxis("追番数（万）",data[5*i:5*i+5][::-1])
            .set_colors(random.choice(["red","#66ccff","#39c5bb","#FFC0CB","#fd96fd"]))
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(title_opts=opts.TitleOpts(title="播放量TOP1-50",title_link='https://www.bilibili.com/v/popular/rank/bangumi',
                                                       subtitle='数据截止到{}'.format(path[-9:-4])
                                                       ,subtitle_link='https://www.bilibili.com/v/popular/rank/bangumi')
                            )
            )
        t.add(c,'{}-{}名'.format(5*i+1,5*(i+1)))
    return t

In [14]:
a=data_resource('./DATA/')
names = set(a['Name'].tolist())
kk=random.choice(list(names))
datas=a.loc[a['Name'] == kk].values.tolist()
datas

[['被解雇的暗黑士兵慢生活的第二人生', '2023-04-22', 1907.6, 46.7],
 ['被解雇的暗黑士兵慢生活的第二人生', '2023-04-23', 1909.5, 46.7],
 ['被解雇的暗黑士兵慢生活的第二人生', '2023-04-24', 1912.7, 46.8]]

In [15]:
from pyecharts.components import Image
from pyecharts.options import ComponentTitleOpts
import json
img = Image()
img.add(
    src='2233.jpg',
    style_opts={"width": "300px", "height": "400px"},
    )
img.set_global_opts(title_opts=ComponentTitleOpts(title='2233'))    

img.render_notebook()


In [23]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line, Pie, Tab, Timeline
def timeline_watch(path) -> Timeline:
    data_watch =sort_watch(path)
    name=[]
    data=[]
    for i in data_watch:
        name.append(i[0])
        data.append(i[1])
    t = Timeline()
    for i in range(0, 10):
        c = (
            Bar(init_opts=opts.InitOpts(width='2000px',
                                       height='1000px',
                                       page_title='4月29日观看总数排行'))
            .add_xaxis([split(name) for name in name[5*i:5*i+5][::-1]])
            .add_yaxis("播放人数（万）",data[5*i:5*i+5][::-1])
            .set_colors(random.choice(["red","#66ccff","#39c5bb","#FFC0CB","#fd96fd"]))
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(title_opts=opts.TitleOpts(title="播放量TOP1-50",title_link='https://www.bilibili.com/v/popular/rank/bangumi',
                                                       subtitle='数据截止到{}'.format(path[-9:-4])
                                                       ,subtitle_link='https://www.bilibili.com/v/popular/rank/bangumi/')
                            )
            )
        t.add(c,'{}-{}名'.format(5*i+1,5*(i+1)))
    return t
def timeline_follow(path) -> Timeline:
    data_follow =sort_follow(path)
    name=[]
    data=[]
    for i in data_follow:
        name.append(i[0])
        data.append(i[1])
    t = Timeline()
    for i in range(0, 10):
        c = (
            Bar()
            .add_xaxis([split(name) for name in name[5*i:5*i+5][::-1]])
            .add_yaxis("追番数（万）",data[5*i:5*i+5][::-1])
            .set_colors(random.choice(["red","#66ccff","#39c5bb","#FFC0CB","#fd96fd"]))
            .reversal_axis()
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
            .set_global_opts(title_opts=opts.TitleOpts(title="播放量TOP1-50",title_link='https://www.bilibili.com/v/popular/rank/bangumi',
                                                       subtitle='数据截止到{}'.format(path[-9:-4])
                                                       ,subtitle_link='https://www.bilibili.com/v/popular/rank/bangumi')
                            )
            )
        t.add(c,'{}-{}名'.format(5*i+1,5*(i+1)))
    return t

def rousen_2233() -> Image:
    img = Image()
    img.add(
        src='2233.jpg',
        style_opts={"width": "300px", "height": "600px"},
    )
    img.set_global_opts(title_opts=ComponentTitleOpts(title='2233'))    
    return img
tab = Tab()
tab.add(rousen_2233(),'可爱的2233~！')
tab.add(timeline_watch('./DATA/2023-04-29.csv'), "播放量榜单")
tab.add(timeline_follow('./DATA/2023-04-29.csv'),'追番人数')
tab
tab.render("TAB.html")

'C:\\Users\\Rousen\\数据可视化\\202012903718石志滨\\dark♂作业\\TAB.html'

In [55]:
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
a=data_resource('./DATA/')
names = set(a['Name'].tolist())
names = list(names)
abc=[]
a=data_resource('./DATA/')
for i in names:    
    datas=a.loc[a['Name'] == i].values.tolist()
    abc.append(datas)
data=[]
for i in abc:
    data.append(i)
QB=[]
for k in data:
    length=len(k)
#     print(k[0])
#     print(k[length-1])
    if length >1:
        data_001=[]
        data_001.append(k[0][0])
        data_001.append(length)
        data_001.append((k[length-1][2]-k[0][2])/length)
        data_001.append((k[length-1][3]-k[0][3])/length)    
        QB.append(data_001)
QB=QB
QB1=fun1(QB)
L=[]
# for i in range(len(QB1)):
#     s=QB1[i][2]+QB1[i][3]*50
#     L.append(s)


[['名侦探柯南（中配）', 8, 125.0, 0.0625], ['间谍过家家', 8, 125.0, 0.025000000000005684], ['博人传 火影忍者新时代', 8, 125.0, 0.02499999999999858], ['名侦探柯南', 8, 125.0, 0.02499999999999858], ['跃动青春', 8, 31.5, 1.1375000000000002], ['Ｄｒ．ＳＴＯＮＥ 石纪元 (第三季)', 8, 39.337500000000006, 0.1875], ['祭品公主与兽之王', 8, 11.6875, 0.6625000000000001], ['蓝色监狱', 8, 24.100000000000023, 0.22499999999999964], ['因为太怕痛就全点防御力了 第二季', 8, 31.325000000000045, 0.07500000000000284], ['放学后失眠的你', 8, 11.274999999999999, 0.42500000000000027], ['虚构推理 第二季', 8, 25.587499999999977, 0.08749999999999147], ['冰剑的魔术师将要统一世界', 8, 15.100000000000001, 0.2124999999999999], ['假面骑士 极狐', 8, 18.01249999999999, 0.08749999999999947], ['国王排名 勇气的宝箱', 8, 28.64999999999999, -0.1875], ['邻家的天使同学', 8, 10.174999999999997, 0.1625000000000001], ['孤独摇滚！', 8, 8.825000000000045, 0.14999999999999858], ['入间同学入魔了 第三季', 8, 15.137499999999989, 0.012499999999995737], ['名侦探柯南：犯人犯泽先生', 8, 13.037500000000001, -0.03750000000000142], ['奇幻世界舅舅', 8, 7.850000000000136, 0.0625], ['文豪野犬 第四季', 8, 7

In [32]:
from pyecharts.components import Image
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Page
from random import choice
from bs4 import BeautifulSoup
switch=0
print('进入循环')
print('刷新网页可以更新数据')
QB1 = fun1(QB)
while(switch==1):
    #数据变化组件
    t = Timeline()
    a=data_resource('./DATA/')
    names = set(a['Name'].tolist())
    kk=choice(list(names))
    datas=a.loc[a['Name'] == kk].values.tolist()
    for i in range(0,2):
        if i == 0:      
            c = (
                Line()
                .add_xaxis([data[1][-5:] for data in datas])  #日期数据【-5：】切割出04-22，节省空间
                .add_yaxis("{}".format(datas[0][0]), [data[2] for data in datas],color='#FFC0CB')  #番名和播放量数据
                .set_global_opts(yaxis_opts=opts.AxisOpts(min_='dataMin'),
                                 title_opts=opts.TitleOpts(title="{}天数据趋势（万）".format(len(datas))))
            )
            t.add(c,'{}'.format('播放量（万）'))
            continue
        if i == 1:      
            c = (
                Line()
                .add_xaxis([data[1][-5:] for data in datas])
                .add_yaxis("{}".format(datas[0][0]), [data[3] for data in datas],color='#66ccff') #番名和追番量数据
                .set_global_opts(title_opts=opts.TitleOpts(title="{}天数据趋势(万)".format(len(datas))),
                                xaxis_opts={"axisLabel": {"textStyle": {"fontSize": 16}}},  # 修改x轴标签字体大小为16
                                 yaxis_opts={"axisLabel": {"textStyle": {"fontSize": 16}}})
                .set_series_opts(label_opts={'fontSize':25})
            )
            t.add(c,'{}'.format('追番数（万）'))

    #图片组件
    image = Image()
    img_src = (
        "./picture/"
        "{}.jpg".format(datas[0][0]))
    image.add(
        src=img_src,
        style_opts={"width": "360px", "height": "480px", "style": "margin-top: 5px"},
    )
    image.set_global_opts({"title":"{}".format(kk), 
                           "title_style":"style='color:#39c5bb'"})
    
    #表格组件
    table = Table()
    headers = ["番名","记录天数","播放量平均增长（万）","追番数平均增长（百）","综合热度评分"]
    table.add(headers, QB1)
    table.set_global_opts(
        title_opts=ComponentTitleOpts(title="综合热度4.22-4.29", subtitle="按综合热度评分降序排序")
    )
    
    page = Page()
    page.add(image,t,table)
    page.add_js_funcs('''alert('刷新可以更新内容')''')
    page.render('测试.html')

    #**************************************************************************************************

    with open('测试.html', 'r', encoding='utf-8') as f:
        r1 = f.read()
    html_bf = BeautifulSoup(r1, 'lxml')
    body = html_bf.find('body').find('p')
    body['style'] = 'font-size:20px;color:#39c5bb;text-align:center '    #字体大小颜色居中
    divs=html_bf.find_all("div")
    divs[1]["style"]="width:360px;height:350px;position:absolute;top:0px;left:50px"
    divs[2]["style"]="width:900px;height:540px;position:absolute;top:15px;left:500px"
    divs[3]["style"]="width:1200px;height:800px;position:absolute;top:610px;left:10px"
    # 将修改后的内容写入文件
    html_new = str(html_bf)
    with open('BI大屏图.html', 'w', encoding='utf-8') as f:
        f.write(html_new)

进入循环
刷新网页可以更新数据


In [33]:
from pyecharts.components import Image
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Page
from random import choice
from bs4 import BeautifulSoup
switch=1
print('进入循环')
print('刷新网页可以更新数据')
while(switch==0):
    #数据变化组件
    t = Timeline()
    a=data_resource('./DATA/')
    names = set(a['Name'].tolist())
    kk=choice(list(names))
    datas=a.loc[a['Name'] == kk].values.tolist()
    for i in range(0,2):
        if i == 0:      
            c = (
                Line()
                .add_xaxis([data[1][-5:] for data in datas])  #日期数据【-5：】切割出04-22，节省空间
                .add_yaxis("{}".format(datas[0][0]), [data[2] for data in datas],color='#FFC0CB')  #番名和播放量数据
                .set_global_opts(yaxis_opts=opts.AxisOpts(min_='dataMin'),
                                 title_opts=opts.TitleOpts(title="{}天数据趋势（万）".format(len(datas))))
            )
            t.add(c,'{}'.format('播放量（万）'))
            continue
        if i == 1:      
            c = (
                Line()
                .add_xaxis([data[1][-5:] for data in datas])
                .add_yaxis("{}".format(datas[0][0]), [data[3] for data in datas],color='#66ccff') #番名和追番量数据
                .set_global_opts(title_opts=opts.TitleOpts(title="{}天数据趋势(万)".format(len(datas))),
                                xaxis_opts={"axisLabel": {"textStyle": {"fontSize": 16}}},  # 修改x轴标签字体大小为16
                                 yaxis_opts={"axisLabel": {"textStyle": {"fontSize": 16}}})
                .set_series_opts(label_opts={'fontSize':25})
            )
            t.add(c,'{}'.format('追番数（万）'))

    #图片组件
    image = Image()
    img_src = (
        "./picture/"
        "{}.jpg".format(datas[0][0]))
    image.add(
        src=img_src,
        style_opts={"width": "360px", "height": "480px", "style": "margin-top: 5px"},
    )
    image.set_global_opts({"title":"{}".format(kk), 
                           "title_style":"style='color:#39c5bb'"})
    print(QB1)
    #表格组件
    table = Table()
    headers = ["番名","记录天数","播放量平均增长（万）","追番数平均增长（百）"]
    table.add(headers, QB1)
    table.set_global_opts(
        title_opts=ComponentTitleOpts(title="综合热度4.22-4.29", subtitle="按综合热度评分降序排序")
    )
    
    page = Page()
    page.add(image,t,table)
    page.add_js_funcs('''alert('刷新可以更新内容')''')
    page.render('测试.html')

    #**************************************************************************************************

    with open('测试.html', 'r', encoding='utf-8') as f:
        r1 = f.read()
    html_bf = BeautifulSoup(r1, 'lxml')
    body = html_bf.find('body').find('p')
    body['style'] = 'font-size:20px;color:#39c5bb;text-align:center '    #字体大小颜色居中
    divs=html_bf.find_all("div")
    divs[1]["style"]="width:360px;height:350px;position:absolute;top:0px;left:50px"
    divs[2]["style"]="width:900px;height:540px;position:absolute;top:15px;left:500px"
    divs[3]["style"]="width:1200px;height:800px;position:absolute;top:610px;left:10px"
    # 将修改后的内容写入文件
    html_new = str(html_bf)
    with open('测试大屏图.html', 'w', encoding='utf-8') as f:
        f.write(html_new)

进入循环
刷新网页可以更新数据


In [16]:
data=data_resource('./DATA/')    #8天50行共400行数据
data
#计算播放量与追番数的平均比值
data1 = data['PV'].tolist()
data2 = data['Follower'].tolist()
i=0
SUM=0
for i in range(len(data1)):
    rate = data1[i]/data2[i]
    #print('{},{}'.format(data1[i],data2[i]))
    SUM = SUM + rate
rate = SUM/len(data1)
print(rate)
#播放量与追番量的比值几乎是50倍
#计算综合热度的时候加权平均.

51.41512736530305


In [112]:
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
a=data_resource('./DATA/')
names = set(a['Name'].tolist())
names = list(names)
abc=[]
a=data_resource('./DATA/')
for i in names:    
    datas=a.loc[a['Name'] == i].values.tolist()
    abc.append(datas)
data=[]
for i in abc:
    data.append(i)
QB=[]
for k in data:
    length=len(k)
#     print(k[0])
#     print(k[length-1])
    if length >1:
        data_001=[]
        data_001.append(k[0][0])
        data_001.append(length)
        data_001.append((k[length-1][2]-k[0][2])/length)
        data_001.append((k[length-1][3]-k[0][3])/length)    
        QB.append(data_001)
QB=QB
print(QB)
# L=[]
# for i in range(len(QB1)):
#     s=QB1[i][2]+QB1[i][3]*50
#     L.append(s)


[['孤独摇滚！', 8, 8.825000000000045, 0.14999999999999858], ['放学后失眠的你', 8, 11.274999999999999, 0.42500000000000027], ['晨曦公主', 2, 2.25, 0.04999999999999716], ['黑色四叶草', 8, 0.0, 0.03750000000000142], ['关于我转生变成史莱姆这档事 第二季', 8, 0.0, 0.012500000000002842], ['物之古物奇谭', 5, 1.4599999999999909, 0.019999999999998862], ['反派大小姐莉泽洛特与实况解说', 3, 1.2666666666666515, 0.033333333333333805], ['咒术回战', 8, 0.0, 0.11249999999998295], ['OVERLORD Ⅲ', 2, 0.0, 0.0], ['间谍过家家', 8, 125.0, 0.025000000000005684], ['火影忍者 疾风传', 8, 0.0, 0.03750000000000142], ['冰剑的魔术师将要统一世界', 8, 15.100000000000001, 0.2124999999999999], ['致不灭的你 第二季', 8, 9.724999999999994, 0.0], ['奇幻世界舅舅', 8, 7.850000000000136, 0.0625], ['假面骑士 极狐', 8, 18.01249999999999, 0.08749999999999947], ['入间同学入魔了 第三季', 8, 15.137499999999989, 0.012499999999995737], ['JOJO的奇妙冒险 黄金之风', 8, 0.0, 0.025000000000005684], ['蓝色监狱', 8, 24.100000000000023, 0.22499999999999964], ['小林家的龙女仆', 8, 0.0, 0.025000000000005684], ['赛马娘 第二季', 8, 5.224999999999966, 0.05000000000000071], ['英雄传说：闪之轨迹',

In [113]:
def apd(QBX):
    for i in range(len(QBX)):
        QBX[i][2]=round(QBX[i][2],4)
        QBX[i][3]=round(QBX[i][3]*100,4)
        QBX[i].append(QBX[i][2]+QBX[i][3]*50/100)
    return QBX
QB20 = fun1(QB)
QB20 = apd(QB20)
print(QB20)
t = Timeline()
a=data_resource('./DATA/')
names = set(a['Name'].tolist())
kk=choice(list(names))
datas=a.loc[a['Name'] == kk].values.tolist()
for i in range(0,2):
    if i == 0:      
        c = (
            Line()
            .add_xaxis([data[1][-5:] for data in datas])  #日期数据【-5：】切割出04-22，节省空间
            .add_yaxis("{}".format(datas[0][0]), [data[2] for data in datas],color='#FFC0CB')  #番名和播放量数据
            .set_global_opts(yaxis_opts=opts.AxisOpts(min_='dataMin'),
                             title_opts=opts.TitleOpts(title="{}天数据趋势（万）".format(len(datas))))
        )
        t.add(c,'{}'.format('播放量（万）'))
        continue
    if i == 1:      
        c = (
            Line()
            .add_xaxis([data[1][-5:] for data in datas])
            .add_yaxis("{}".format(datas[0][0]), [data[3] for data in datas],color='#66ccff') #番名和追番量数据
            .set_global_opts(title_opts=opts.TitleOpts(title="{}天数据趋势(万)".format(len(datas))),
                            xaxis_opts={"axisLabel": {"textStyle": {"fontSize": 16}}},  # 修改x轴标签字体大小为16
                             yaxis_opts={"axisLabel": {"textStyle": {"fontSize": 16}}})
            .set_series_opts(label_opts={'fontSize':25})
        )
        t.add(c,'{}'.format('追番数（万）'))

#图片组件
image = Image()
img_src = (
    "./picture/"
    "{}.jpg".format(datas[0][0]))
image.add(
    src=img_src,
    style_opts={"width": "360px", "height": "480px", "style": "margin-top: 5px"},
)
image.set_global_opts({"title":"{}".format(kk), 
                       "title_style":"style='color:#39c5bb'"})

#表格组件
table = Table()
headers = ["番名","记录天数","播放量平均增长（万）","追番数平均增长（百）","综合热度评分"]
table.add(headers, QB20)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title="综合热度4.22-4.29", subtitle="按综合热度评分降序排序")
)

[['名侦探柯南（中配）', 8, 125.0, 6.25, 128.125], ['间谍过家家', 8, 125.0, 2.5, 126.25], ['博人传 火影忍者新时代', 8, 125.0, 2.5, 126.25], ['名侦探柯南', 8, 125.0, 2.5, 126.25], ['跃动青春', 8, 31.5, 113.75, 88.375], ['Ｄｒ．ＳＴＯＮＥ 石纪元 (第三季)', 8, 39.3375, 18.75, 48.7125], ['祭品公主与兽之王', 8, 11.6875, 66.25, 44.8125], ['蓝色监狱', 8, 24.1, 22.5, 35.35], ['因为太怕痛就全点防御力了 第二季', 8, 31.325, 7.5, 35.075], ['放学后失眠的你', 8, 11.275, 42.5, 32.525], ['虚构推理 第二季', 8, 25.5875, 8.75, 29.9625], ['冰剑的魔术师将要统一世界', 8, 15.1, 21.25, 25.725], ['假面骑士 极狐', 8, 18.0125, 8.75, 22.3875], ['国王排名 勇气的宝箱', 8, 28.65, -18.75, 19.275], ['邻家的天使同学', 8, 10.175, 16.25, 18.3], ['孤独摇滚！', 8, 8.825, 15.0, 16.325], ['入间同学入魔了 第三季', 8, 15.1375, 1.25, 15.7625], ['名侦探柯南：犯人犯泽先生', 8, 13.0375, -3.75, 11.1625], ['奇幻世界舅舅', 8, 7.85, 6.25, 10.975], ['文豪野犬 第四季', 8, 7.35, 5.0, 9.85], ['致不灭的你 第二季', 8, 9.725, 0.0, 9.725], ['鬼灭之刃', 8, 0.0, 18.75, 9.375], ['见面5秒开始战斗', 8, 5.475, 7.5, 9.225], ['赛马娘 第二季', 8, 5.225, 5.0, 7.725], ['久保同学不放过我', 8, 2.6875, 6.25, 5.8125], ['四月是你的谎言', 8, 0.0, 11.25, 5.62

In [114]:
QB2=fun2(QB)   
QB2=QB2[::-1]         #追番前四
data=data_resource('./DATA/')
img1=Image_atd(QB2[0])
img2=Image_atd(QB2[1])
img3=Image_atd(QB2[2])
img4=Image_atd(QB2[3])
datas0=data.loc[data['Name'] == QB2[0][0]].values.tolist()
datas1=data.loc[data['Name'] == QB2[1][0]].values.tolist()
datas2=data.loc[data['Name'] == QB2[2][0]].values.tolist()
datas3=data.loc[data['Name'] == QB2[3][0]].values.tolist()
l1=Line_atd(datas0)
l2=Line_atd(datas1)
l3=Line_atd(datas2)
l4=Line_atd(datas3)
t1 = timeline_watch('./DATA/2023-04-29.csv')
t2 = timeline_follow('./DATA/2023-04-29.csv')
page = Page()
page.add(img1,img2,img3,img4,l1,l2,l3,l4,t1,t2,image,t,table)
page.render('前四.html')

#*******************************************************
with open('前四.html', 'r', encoding='utf-8') as f:
    r1 = f.read()
html_bf = BeautifulSoup(r1, 'lxml')
bodys = html_bf.find('body').find_all('p')
for body in bodys:
    body['style'] = 'font-size:20px;color:#39c5bb;text-align:center '
body = html_bf.find('body')
body['style'] = 'background-image:url(贴窗花-rafa.jpg);background-repeat: no-repeat;background-attachment: fixed;background-size: 100% 100%;'
divs=html_bf.find_all("div")
divs[1]["style"]="width:300px;height:380px;position:absolute;top:70px;left:70px"
divs[2]["style"]="width:300px;height:380px;position:absolute;top:70px;left:418px"
divs[3]["style"]="width:300px;height:380px;position:absolute;top:70px;left:766px"
divs[4]["style"]="width:300px;height:380px;position:absolute;top:70px;left:1114px"
divs[5]["style"]="width:780px;height:380px;position:absolute;top:1200px;left:0px;background-color:#aaff66"
divs[6]["style"]="width:780px;height:380px;position:absolute;top:1200px;left:780px;background-color:#aaff66"
divs[7]["style"]="width:780px;height:380px;position:absolute;top:1550px;left:0px;background-color:#aaff66"
divs[8]["style"]="width:780px;height:380px;position:absolute;top:1550px;left:780px;background-color:#aaff66"
divs[9]["style"]="width:780px;height:380px;position:absolute;top:1930px;left:0px;background-color:#FFFFFF"
divs[10]["style"]="width:780px;height:380px;position:absolute;top:1930px;left:780px;background-color:#FFFFFF"
divs[11]["style"]="width:420px;height:380px;position:absolute;top:630px;left:80px;"
divs[12]["style"]="width:900px;height:540px;position:absolute;top:650px;left:460px;background-color:#FFFFFF"
divs[13]["style"]="width:1500px;height:3200px;position:absolute;top:2310px;left:10px;background-color:#FFFFFF"


div_title = "<div align=\"center\" style=\"width:1400px;left:200px;\">\n<span style=\"font-size:40px;color:#66ccff\"><b>BI大屏图(202012903718-石志滨)</b></div>"
body.insert(0, BeautifulSoup(div_title, "lxml").div)
# 将修改后的内容写入文件
html_new = str(html_bf)
with open('前四！.html', 'w', encoding='utf-8') as f:
    f.write(html_new)

In [21]:
a=data_resource('./DATA/')    #8天50行共400行数据
names = set(a['Name'].tolist())
kk=random.choice(list(names))
datas=data.loc[a['Name'] == kk].values.tolist()
print(datas)
a

[['致不灭的你 第二季', '2023-04-22', 1947.4, 483.0], ['致不灭的你 第二季', '2023-04-23', 1952.1, 483.0], ['致不灭的你 第二季', '2023-04-24', 1973.8, 483.0], ['致不灭的你 第二季', '2023-04-25', 1995.1, 483.0], ['致不灭的你 第二季', '2023-04-26', 2002.4, 483.0], ['致不灭的你 第二季', '2023-04-27', 2010.2, 483.0], ['致不灭的你 第二季', '2023-04-28', 2019.6, 483.0], ['致不灭的你 第二季', '2023-04-29', 2025.2, 483.0]]


,Name,Date,PV,Follower
0,英雄传说：闪之轨迹,2023-04-23,215.0,20.3
1,英雄传说：闪之轨迹,2023-04-24,216.5,20.3
2,英雄传说：闪之轨迹,2023-04-25,217.9,20.3
3,夏日重现,2023-04-25,21000.0,431.1
4,夏日重现,2023-04-26,21000.0,431.2
...,...,...,...,...
395,孤独摇滚！,2023-04-28,9458.5,266.9
396,孤独摇滚！,2023-04-29,9467.6,267.0
397,因为太怕痛就全点防御力了,2023-04-22,27000.0,659.0
398,因为太怕痛就全点防御力了,2023-04-26,27000.0,659.2
